In [1]:
!pip install textblob
!python -m textblob.download_corpora lite

[nltk_data] Downloading package brown to /home/jovyan/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Finished.


In [2]:
import json

import pyspark
import pyspark.sql
from pyspark.ml import Pipeline

from modules.posts import SentenceTransformer, PostTransformer, TranslateTransformer, SpeechPartsTransformer, SentimentTransformer


sc = pyspark.SparkContext('local[*]', 'PipelineFlow')
sess = pyspark.sql.SparkSession(sc)
rdd = sc.wholeTextFiles('data/*')
rdd = rdd.map(lambda x: (x[0], json.loads(x[1])))
df = rdd.toDF(['file', 'content'])

poster = PostTransformer().setInputCol('content').setOutputCol('posts')
translator = TranslateTransformer().setInputCol('posts').setOutputCol('translated')
sentencer = SentenceTransformer().setInputCol('translated').setOutputCol('sentences')
speechParter = SpeechPartsTransformer().setInputCol('translated').setOutputCol('speechParts')
sentimenter = SentimentTransformer().setInputCol('translated').setOutputCol('sentiments')

pipeline = Pipeline(stages=[poster, translator, sentencer, speechParter, sentimenter])
out = pipeline.fit(df).transform(df)

a = out.select('sentences').first().sentences[0]
b = out.select('sentences').first().sentences[1]
c = out.select('sentences').first().sentences[2]
d = out.select('translated').first().translated[0]
e = out.select('speechParts').first().speechParts
f = out.select('sentiments').first().sentiments[0]
g = out.select('sentiments').first().sentiments[1]
h = out.select('sentiments').first().sentiments[2]

print('{}\n\n{}\n\n{}\n\n{}\n\n{}\n\n{}\n\n{}\n\n{}'.format(a,b,c,d,e,f,g,h))

Oh, please ... very interesting product.

I've never had contact with such paint.

Regards ;)

Please ... very interesting product. I never had contact with such a paint. Regards ;)

{'NN': 33, 'JJ': 18, 'CC': 5, 'CD': 1, 'VBN': 1, 'IN': 20, 'VBP': 9, 'WDT': 1, 'VB': 12, 'PRP': 28, 'VBZ': 11, 'DT': 22, 'RB': 15, 'NNP': 2, 'NNS': 11, 'JJR': 1, 'UH': 1, 'EX': 1, 'PRP$': 3, 'MD': 5, 'VBD': 7, 'WP': 3, 'VBG': 1, 'TO': 4}

Row(polarity=0.3, subjectivity=0.7166666666666667)

Row(polarity=0.16666666666666666, subjectivity=0.3666666666666667)

Row(polarity=-0.7999999999999999, subjectivity=1.0)
